# Computer Use AI Agent

In [ ]:
# import necessary libraries 

## 1. Request a Prompt from the User

By prompting the user at runtime, we can create a more flexible workflow.

In [2]:
user_task = input("👋 Hi there! What task can I help you perform today? \n>> ")

## 2. Convert the Task into a Series of Actions

This is where your reasoning model comes in! By feeding it the `user-input` string, along with a prompt that provides clear instructions to the model about for generating a plan. These are the essential components of the prompt: 
* Interpret the user's task
* Break it into a sequence of essential steps
* Be concise! Take the path of least resistance
* Detect if there is a login required

## 3. Execute the AI-Generated Plan Using `browser-use`

We can now pass the plan to the `Agent` class from the `browser-use` library. This agent will spin up a browser (typically this uses [Puppeteer](https://pptr.dev/) or [Playwright](https://playwright.dev/) under the hood).

This agent is able to perform many basic actions, including: 
* clicking buttons
* typing into forms
* opening and closing tabs

In [ ]:
agent = Agent(
    task = task_plan,   # the plan generated by our GPT-4o model
    llm = llm,          # our GPT-4o model 
)
await agent.run()

## 4. Implement Re-try Logic

This step is intended to catch any instabilities in the web pages the agent is trying to access. For example, a button that needs to be clicked won't load, or pop-ups distract from the main task. 

Whenever a step fails, it will be retried 3 times before aborting the task. 

In [ ]:
max_retries = 3
while retries < max_retries:
    try:
        # Execute the task
        await agent.run()
        print(f"✅ Task completed successfully")
        break
    except Exception as e:
        retries += 1
        print(f"⚠️ Attempt {retries}/{max_retries} failed: {e}")
        if retries == max_retries:
            print("❌ All attempts failed. Please check the task and try again.")

## 5. Safeguard For Handling Login Pages

### There are two options for adhering to best practices in data privacy: 
1. Implement [WAIT] placeholders in a cloud-based LLM ⛅️
2. Fully automated Local LLM that can access user credentials 🔐

|        | Benefits | Drawbacks |
|:------ |:---------|:----------|  
| Cloud-Based Approach| * Simple setup on any machine | * Workflow is interrupted and requires manual user intervention at sensitive steps |
|        | * Can easily try different models and upgrade to the latest release | * Incurs usage costs based on the API calls and model pricing |
| Local LLM Approach | * Complete E2E automation without user intervention | * Requires significant local hardware resources | 
|   | * Full privacy protection for all user data | * Model quality may be lower than cloud alternatives due to compute restraints |
|   | * No ongoing costs can can run without internet after initial setup | * More complex setup and maintenance | 
|   | * Can integrate deeply with system security features | * Higher technical knowledge required for proper security configurations |   

For this simple architecture, we will be using the cloud-based approach that implements breaks in the workflow for the user to securely enter their login information. We do this by designing the system to: 
* Detect when a login might be required (performed during planning by our GPT-4o model)
* Pause the agent and wait for the user to manually enter their credentials
* Resume the process and continue with the remaining steps